<a href="https://colab.research.google.com/github/faithfulness121/beginner-project/blob/main/Electriv_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib  as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
!pip install ydata-profiling
from ydata_profiling import ProfileReport
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#loading dataset
df= pd.read_csv('/content//Electric_cars_dataset.csv')
df.head()

In [ ]:
#explorring data
print("\--- Dataset Info ---")
print(df.info())

print("\n--- Dataset Description ---")
print(df.describe())

print("\n--- Duplicates ---")
print("Duplicates:", df.duplicated().sum())

In [ ]:
columns = df.columns.tolist()
print(columns)

In [ ]:
profile =ProfileReport(df, title='Electric car', explorative=True)
profile.to_notebook_iframe()

In [ ]:
df = df.drop_duplicates()
df = df.drop(columns=['ID', 'VIN (1-10)', 'DOL Vehicle ID', 'Vehicle Location'])


In [ ]:
# Display missing values
missing = df.isnull().sum()

df['Expected Price ($1k)'] = pd.to_numeric(df['Expected Price ($1k)'], errors='coerce')

# Drop rows with missing target
df = df.dropna(subset=['Expected Price ($1k)'])

In [ ]:
# Separate numerical and categorical columns
numerical_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Fill numerical missing values with median
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Fill categorical missing values with mode or 'Unknown'
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode = df[col].mode()
        if not mode.empty:
            df[col].fillna(mode[0], inplace=True)
        else:
            df[col].fillna('Unknown', inplace=True)

# Display missing values after handling
missing_after = df.isnull().sum()

missing, missing_after




In [ ]:
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])


In [ ]:
# Handle outliers using IQR method

# Select only numerical columns for outlier detection
numerical_df = df.select_dtypes(include=np.number)

Q1 = numerical_df.quantile(0.25)
Q3 = numerical_df.quantile(0.75)
IQR = Q3 - Q1

# Create a boolean mask for outliers in numerical columns
outlier_mask = ((numerical_df < (Q1 - 1.5 * IQR)) | (numerical_df > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter the original DataFrame using the boolean mask
df = df[outlier_mask]

display(df.head())

In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)

# Select features and target
X = df_encoded.drop(columns=['Expected Price ($1k)'])
y = df_encoded['Expected Price ($1k)']


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardizing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#MODEL
# Use SVR for regression
svm_model = SVR(kernel='linear')
svm_model.fit(X_train_scaled, y_train)

# Evaluate the Model
y_pred = svm_model.predict(X_test_scaled)

# Use regression evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n--- Evaluation Metrics ---")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

In [77]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
from sklearn.svm import SVR

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.1],
    'epsilon': [0.1, 0.5, 1.0],
    'kernel': ['rbf']
}

# Grid Search with Cross-Validation
grid_search = GridSearchCV(SVR(), param_grid, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best Parameters and Score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation R² Score:", grid_search.best_score_)

# Evaluate on Test Set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nTest Set Evaluation:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'C': 10, 'epsilon': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
Best Cross-Validation R² Score: 0.6389165814051151

Test Set Evaluation:
MAE: 5.082624269175062
RMSE: 11.70088113511914
R²: 0.751892891052353
